In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")

# =============== Utilities ===============

def to_binary(vec):
    return (1 / (1 + np.exp(-10 * (vec - 0.5))) >= 0.5).astype(int)

def compute_accuracy_and_mse(X, y, mask, k=5, cv=10, seed=None):
    """
    用於：
      - cost_function: 使用 acc（不動）
      - evaluate_final: 使用 MSE（符合論文 Eq.15）
    """
    y_enc = pd.factorize(y)[0]

    X_sel = X[:, mask.astype(bool)]
    skf = StratifiedKFold(
        n_splits=min(cv, len(np.unique(y_enc))),
        shuffle=True,
        random_state=seed
    )

    clf = KNeighborsClassifier(n_neighbors=k)
    y_true, y_pred = [], []

    for tr, te in skf.split(X_sel, y_enc):
        clf.fit(X_sel[tr], y_enc[tr])
        pred = clf.predict(X_sel[te])
        y_true.extend(y_enc[te])
        y_pred.extend(pred)

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    acc = np.mean(y_true == y_pred)
    mse = mean_squared_error(y_true, y_pred)

    return acc, mse


# =============== (Eq.12)(Eq.13) cost/fit 公式 — 完全不動 ===============

def cost_function(mask, X, y, omega=0.99, k=5, cv=10, seed=None):
    D = X.shape[1]
    mask = mask.astype(int).copy()

    # 保證至少有一個特徵
    if mask.sum() == 0:
        mask[np.argmax(np.var(X, axis=0))] = 1

    sel_frac = mask.sum() / D
    acc, _ = compute_accuracy_and_mse(X, y, mask, k, cv, seed)

    err = 1 - acc                      # 分類錯誤率
    cost = omega * err + (1-omega) * sel_frac

    # 如果你還想保留 "fit" 這個輸出，就定義成 1 - cost 純粹拿來看數字
    fit = 1 - cost

    return cost, fit, acc, sel_frac

# =============== 最終 Fitness(F) — Eq.(15) 用 MSE ===============

def evaluate_final(mask, X, y, h1=0.99, h2=0.01, k=5, cv=10, seed=None):
    """
    F = h1*Select + h2*MSE
    完全照論文 Eq.(15)
    """
    D = X.shape[1]

    # 先複製並保證至少一個特徵
    mask = mask.astype(int).copy()
    if mask.sum() == 0:
        mask[np.argmax(np.var(X, axis=0))] = 1

    sel_frac = mask.sum() / D
    acc, mse = compute_accuracy_and_mse(X, y, mask, k, cv, seed)

    F = h1 * sel_frac + h2 * mse
    return F, mse, sel_frac, acc



# =============== bWWPA Algorithm（你的版本，不動 update） ===============

class bWWPA:
    def __init__(self, pop=10, dim=None, iters=100, omega=0.99, k=5, cv=10, seed=None):
        self.pop = pop
        self.dim = dim
        self.iters = iters
        self.omega = omega
        self.k = k
        self.cv = cv
        self.rng = np.random.default_rng(seed)
        self.lb, self.ub = 0, 1

    def initialize(self, X):
        if self.dim is None:
            self.dim = X.shape[1]
        self.P = self.rng.random((self.pop, self.dim))
        self.K = self.rng.random(self.pop)
        # 現在我們 minimize J，因此初始化成 +∞
        self.best_cost = float('inf')
        self.best_sol = None


    def evaluate(self, X, y, seed=None):
        vals = np.zeros(self.pop)

        for i in range(self.pop):
            mask = to_binary(self.P[i])
            cost, fit, acc, sel = cost_function(
                mask, X, y,
                omega=self.omega, k=self.k, cv=self.cv,
                seed=seed
            )
            vals[i] = cost   # ← 這代的是 cost，不再是 J

            # bWWPA: minimize cost
            if cost < self.best_cost:
                self.best_cost = cost
                self.best_sol = self.P[i].copy()

        return vals

    def run(self, X, y, run_seed=0):
        self.initialize(X)
        self.evaluate(X, y, seed=run_seed)

        history = []
        for t in range(1, self.iters + 1):

            F_vals = self.rng.uniform(-1, 1, self.pop)
            C_vals = self.rng.uniform(-1, 1, self.pop)

            for i in range(self.pop):
                r1, r2, r3, r = self.rng.random(4)
                Ki, Fi, Ci = self.K[i], F_vals[i], C_vals[i]

                # 你的更新公式 — 完全不動
                if r < 0.5:
                    W = r1 * (self.P[i] + 2*Ki)
                    self.P[i] += W * (2*Ki + r2)
                else:
                    best = self.best_sol if self.best_sol is not None else self.P[i]
                    W = r3 * (Ki * best + r3 * self.P[i])
                    self.P[i] += Ki * W

                if self.rng.random() < 0.01:
                    theta = self.rng.uniform(0, 2*np.pi, self.dim)
                    self.P[i] = (r1 + Ki) * np.sin((Fi / (Ci + 1e-8)) * theta)

                self.P[i] = np.clip(self.P[i], self.lb, self.ub)

            self.K = np.clip(1 + 2*(t**2)/(self.iters**3) + F_vals, 0, 1)

            vals = self.evaluate(X, y, seed=run_seed)
            history.append(self.best_cost)

        return history, to_binary(self.best_sol)


# =============== Experiment Runner ===============

def run_experiment(X, y, name,
                   runs=20, pop=10, iters=100,
                   omega=0.99, h1=0.99, h2=0.01,
                   k=5, cv=10):

    print(f"\n=== {name} Dataset ===")
    print(f"runs={runs}, pop={pop}, iters={iters}, omega={omega}\n")

    all_F, all_mse, all_sel, all_acc, all_cost = [], [], [], [], []

    for r in range(runs):
        algo = bWWPA(pop, X.shape[1], iters, omega, k, cv, seed=r)
        _, best_mask = algo.run(X, y, run_seed=r)

        # 最終 Fitness (Eq.15)
        F, mse, sel, acc = evaluate_final(best_mask, X, y, h1, h2, k, cv, seed=r)

        # cost (Eq.13)
        cost, fit_val, acc2, sel2 = cost_function(best_mask, X, y, omega, k, cv, seed=r)

        all_F.append(F)
        all_mse.append(mse)
        all_sel.append(sel)
        all_acc.append(acc)
        all_cost.append(cost)

        print(f"Run {r+1:02d}:  F={F:.4f},  cost={cost:.4f},  err(MSE)={mse:.4f},  acc={acc:.4f},  select={sel:.4f}")

    print(f"\n--- {name} Summary ---")
    print(f"Average Fitness(F, Eq.15) : {np.mean(all_F):.4f}")
    print(f"Average Cost(1-fit)       : {np.mean(all_cost):.4f}")
    print(f"Average MSE               : {np.mean(all_mse):.4f}")
    print(f"Average Error(1-acc)      : {np.mean([1-x for x in all_acc]):.4f}")
    print(f"Average Select Ratio      : {np.mean(all_sel):.4f}")


# =============== Main ===============

if __name__ == "__main__":

    # Zoo dataset
    df_zoo = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/zoo/zoo.data", header=None)
    X_zoo = MinMaxScaler().fit_transform(df_zoo.iloc[:, 1:-1])
    y_zoo = pd.factorize(df_zoo.iloc[:, -1])[0]

    run_experiment(
        X_zoo, y_zoo, "Zoo",
        runs=20, pop=20, iters=100,
        omega=0.9,
        h1=0.99, h2=0.01,
        k=5, cv=5,
    )

    # Sonar dataset
    df_sonar = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data", header=None)
    X_sonar = MinMaxScaler().fit_transform(df_sonar.iloc[:, :-1])
    y_sonar = np.where(df_sonar.iloc[:, -1] == "R", 0, 1)

    run_experiment(
        X_sonar, y_sonar, "Sonar",
        runs=30, pop=20, iters=200,
        omega=0.95,
        h1=0.99, h2=0.01,
        k=9, cv=10,
    )


=== Zoo Dataset ===
runs=20, pop=20, iters=100, omega=0.9

Run 01:  F=0.4376,  cost=0.1061,  err(MSE)=0.4455,  acc=0.9307,  select=0.4375
Run 02:  F=0.5614,  cost=0.1186,  err(MSE)=0.4554,  acc=0.9307,  select=0.5625
Run 03:  F=0.6257,  cost=0.1071,  err(MSE)=0.6931,  acc=0.9505,  select=0.6250
Run 04:  F=0.3769,  cost=0.0910,  err(MSE)=0.5644,  acc=0.9406,  select=0.3750
Run 05:  F=0.4393,  cost=0.1150,  err(MSE)=0.6139,  acc=0.9208,  select=0.4375
Run 06:  F=0.4982,  cost=0.1213,  err(MSE)=0.3168,  acc=0.9208,  select=0.5000
Run 07:  F=0.6834,  cost=0.1044,  err(MSE)=0.2772,  acc=0.9604,  select=0.6875
Run 08:  F=0.4463,  cost=0.1061,  err(MSE)=1.3168,  acc=0.9307,  select=0.4375
Run 09:  F=0.5605,  cost=0.1008,  err(MSE)=0.3663,  acc=0.9505,  select=0.5625
Run 10:  F=0.6269,  cost=0.1338,  err(MSE)=0.8119,  acc=0.9208,  select=0.6250
Run 11:  F=0.5611,  cost=0.0919,  err(MSE)=0.4257,  acc=0.9604,  select=0.5625
Run 12:  F=0.5666,  cost=0.0919,  err(MSE)=0.9703,  acc=0.9604,  select